# Soft SVM (Support vector machine) using convex Quadratic program 

## Step 1: Import All required Libraries

In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers as cvxopt_solvers

## Step 2: Read CSV file (For this i have uploded the file on google colab) 

In [15]:
df = pd.read_csv('file.csv')
print(df.shape,df)

(2665, 6)       Temperature   Humidity       Light          CO2  HumidityRatio  Occupancy
0       23.700000  26.272000  585.200000   749.200000       0.004764          1
1       23.718000  26.290000  578.400000   760.400000       0.004773          1
2       23.730000  26.230000  572.666667   769.666667       0.004765          1
3       23.722500  26.125000  493.750000   774.750000       0.004744          1
4       23.754000  26.200000  488.600000   779.000000       0.004767          1
...           ...        ...         ...          ...            ...        ...
2660    24.290000  25.700000  808.000000  1150.250000       0.004829          1
2661    24.330000  25.736000  809.800000  1129.200000       0.004848          1
2662    24.330000  25.700000  817.000000  1125.800000       0.004841          1
2663    24.356667  25.700000  813.000000  1123.000000       0.004849          1
2664    24.408333  25.681667  798.000000  1124.000000       0.004860          1

[2665 rows x 6 columns]


## Step 3: Store the features and target in X , Y seperately and since target has value 0,1 change label 0 to -1 

In [16]:
X = df.iloc[:,:-1].values
Y = df.iloc[:,-1:].values
print(X.shape,X)
Y[Y==0]=-1
print(Y.shape,Y)

(2665, 5) [[2.37000000e+01 2.62720000e+01 5.85200000e+02 7.49200000e+02
  4.76416302e-03]
 [2.37180000e+01 2.62900000e+01 5.78400000e+02 7.60400000e+02
  4.77266099e-03]
 [2.37300000e+01 2.62300000e+01 5.72666667e+02 7.69666667e+02
  4.76515255e-03]
 ...
 [2.43300000e+01 2.57000000e+01 8.17000000e+02 1.12580000e+03
  4.84075873e-03]
 [2.43566667e+01 2.57000000e+01 8.13000000e+02 1.12300000e+03
  4.84855928e-03]
 [2.44083333e+01 2.56816667e+01 7.98000000e+02 1.12400000e+03
  4.86020770e-03]]
(2665, 1) [[1]
 [1]
 [1]
 ...
 [1]
 [1]
 [1]]


## Step 4: Processing the Data and Splitting


In [17]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
X_scale = min_max_scaler.fit_transform(X)
X_scale

array([[0.83168317, 0.44513204, 0.34479305, 0.33003334, 0.70421197],
       [0.8359604 , 0.44705255, 0.34078657, 0.34152347, 0.70830847],
       [0.83881188, 0.44065084, 0.33740855, 0.35103018, 0.70468898],
       ...,
       [0.98138614, 0.38410243, 0.48136692, 0.71638882, 0.74113545],
       [0.98772277, 0.38410243, 0.47901016, 0.71351629, 0.74489576],
       [1.        , 0.38214635, 0.47017234, 0.71454219, 0.75051096]])

A. Split the data in 70,30 for training and testing the model 

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X_scale, Y, test_size=0.3, random_state=1)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1865, 5) (800, 5) (1865, 1) (800, 1)


B. Split the data in 80,20 for training and testing the model 

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X_scale, Y, test_size=0.2, random_state=1)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(2132, 5) (533, 5) (2132, 1) (533, 1)


C. Split the data in 90,10 for training and testing the model 

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X_scale, Y, test_size=0.1, random_state=1)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(2398, 5) (267, 5) (2398, 1) (267, 1)


## Step 5: Optimizing the weight vector w using the cvxopt_solvers
## Here we need to change the variables as:
### max(a) 1/2*a^T*H^T*a - 1^T*a
### such that -a<=0 and y^T*a=0
### where for the Quadratic solver the respective variables are:
### X=a , P=H (matrix of size m×m), q=-1 ( a vector of size m×1), G=-a (matrix of size 2m×m, such that a diagonal matrix of -1s of size m×m is concatenated vertically with another diagonal matrix of 1s of size m×m) , h=0 (a vector of size 2m×1, with zeros in the first m cells and C in the other m cells) , A=y (the label vector of size m×1) , b=0 , H = ghram matrix
## Also for regularization c=10 is used.

In [21]:
#Initializing values and computing H. Note the 1.0 to force to float type
c=10
samples,features = X_train.shape
y = y_train.reshape(-1,1) * 1.0
X_dash = y*X_train
H = np.dot(X_dash , X_dash.T) * 1.0

#Converting into cvxopt format
P = cvxopt_matrix(H)
q = cvxopt_matrix(-np.ones((samples, 1)))
G = cvxopt_matrix(np.vstack((np.eye(samples)*-1,np.eye(samples))))
h = cvxopt_matrix(np.hstack((np.zeros(samples), np.ones(samples)))*c)
A = cvxopt_matrix(y.reshape(1, -1))
b = cvxopt_matrix(np.zeros(1))

## Step 6: Calculating optimized value of w using the defined solver and checking for alphas. 

In [22]:
sol = cvxopt_solvers.qp(P, q, G, h, A, b)
alphas = np.array(sol['x'])

     pcost       dcost       gap    pres   dres
 0: -3.1979e+01 -4.9545e+05  3e+06  3e+00  7e-14
 1:  4.2303e+03 -2.9511e+05  5e+05  3e-01  6e-14
 2:  4.5119e+03 -4.3606e+04  6e+04  2e-02  7e-14
 3:  7.1618e+02 -1.0823e+04  1e+04  3e-03  5e-14
 4: -2.5883e+02 -6.3464e+03  6e+03  1e-03  5e-14
 5: -9.2517e+02 -2.2303e+03  1e+03  2e-04  6e-14
 6: -9.6597e+02 -2.0619e+03  1e+03  1e-04  5e-14
 7: -9.6082e+02 -2.0555e+03  1e+03  1e-04  5e-14
 8: -9.6982e+02 -2.0637e+03  1e+03  1e-04  5e-14
 9: -1.0209e+03 -1.7639e+03  7e+02  2e-05  6e-14
10: -1.0523e+03 -1.5596e+03  5e+02  1e-05  5e-14
11: -1.0567e+03 -1.5614e+03  5e+02  1e-05  5e-14
12: -1.0738e+03 -1.5299e+03  5e+02  1e-05  5e-14
13: -1.0728e+03 -1.5181e+03  4e+02  9e-06  6e-14
14: -1.0989e+03 -1.4268e+03  3e+02  5e-06  6e-14
15: -1.1162e+03 -1.3853e+03  3e+02  2e-06  6e-14
16: -1.1236e+03 -1.3648e+03  2e+02  2e-06  6e-14
17: -1.1279e+03 -1.3394e+03  2e+02  6e-07  6e-14
18: -1.1504e+03 -1.2776e+03  1e+02  2e-07  7e-14
19: -1.1572e+03 -1.26

## Here W = alphai*yi*xi
## Also b = 1/n(sum(y[i]-W*xi))

In [23]:
# w parameters calculated using the alphas which we get after the solver gives output.
w = np.dot((y_train * alphas).T,X_train).reshape(-1,1)

# Computing b
is_sv = (alphas > 1e-4).flatten()
b = y[is_sv] - np.dot(X_train[is_sv], w)
b = sum(b)/len(b)

# Display results
print('Alphas = ',alphas[alphas > 1e-4])
print('w = ', w.flatten())
print('b = ', b)

Alphas =  [9.99999501e+00 9.99999993e+00 9.99985853e+00 9.99998891e+00
 9.99999670e+00 9.99998660e+00 9.99999996e+00 9.99999994e+00
 9.99998511e+00 9.99985346e+00 9.99999845e+00 9.99999993e+00
 9.99999979e+00 9.99999699e+00 9.99999551e+00 9.99999996e+00
 9.99999993e+00 9.99999993e+00 9.99999994e+00 9.22587434e+00
 9.99999994e+00 9.99998762e+00 9.99999993e+00 9.99999810e+00
 9.99999994e+00 9.99999993e+00 9.99999712e+00 9.99999996e+00
 9.99999731e+00 9.99999874e+00 9.99999993e+00 9.99999584e+00
 9.99999717e+00 9.99999811e+00 9.99999994e+00 9.99993809e+00
 9.99999993e+00 9.99999815e+00 9.99999497e+00 9.99999996e+00
 9.99999996e+00 9.99999667e+00 9.99998189e+00 9.99999857e+00
 9.99999644e+00 9.99999787e+00 9.99999994e+00 9.99999996e+00
 9.99999993e+00 9.99999991e+00 9.99999993e+00 9.99999541e+00
 9.99999805e+00 4.71174382e+00 9.99999145e+00 9.99999761e+00
 9.99999624e+00 9.99999993e+00 9.99999993e+00 9.99999996e+00
 9.99999994e+00 8.59501817e+00 9.99999995e+00 9.99999994e+00
 9.99999993e+0

In [24]:
X_Support_vectors = X_train[is_sv]
y_support_vectors = y[is_sv]
print(alphas,'\n',"Support vectors are : = ",X_Support_vectors,'\n','\n',"Number of support vectors = ",len(X_Support_vectors))

[[1.69269980e-07]
 [2.61525308e-06]
 [1.21061967e-07]
 ...
 [1.17189763e-06]
 [1.10097704e-07]
 [1.41947323e-06]] 
 Support vectors are : =  [[0.56079208 0.31154975 0.25511857 0.41369582 0.4473341 ]
 [0.5560396  0.32648706 0.25181912 0.43713773 0.45631814]
 [0.59405941 0.98052814 0.25511857 0.98999744 0.98483829]
 [0.7299802  0.39178448 0.10487553 0.56127212 0.60235773]
 [0.57029703 0.30941584 0.25511857 0.42010772 0.45068295]
 [0.63524752 0.58682315 0.25276182 0.64519107 0.70278279]
 [0.47524752 0.41611096 0.35837384 0.53193126 0.48155751]
 [0.23762376 0.34035743 0.27338341 0.3593229  0.30416772]
 [0.73425743 0.38650307 0.10222419 0.54039497 0.60051181]
 [0.72356436 0.39743932 0.10487553 0.57091562 0.60329369]
 [0.52039604 0.27923002 0.24686994 0.35437292 0.40162559]
 [0.52693069 0.27980795 0.24686994 0.37163375 0.40542571]
 [0.04514851 0.17071219 0.18367948 0.04847397 0.09731328]
 [0.57029703 0.32719836 0.25511857 0.43789006 0.46439684]
 [0.57029703 0.30941584 0.25511857 0.4149782  0

## Step 7: Creating target Array and calculating the target values.

In [25]:
print(X_test.shape,w.shape)
target = np.where(np.dot(X_test,w)+b>=0.0,1,-1)
t = np.where(np.dot(X_train,w)+b>=0.0,1,-1)
print(target.flatten(),'\n',t.flatten(),'\n',y_test.flatten())

(267, 5) (5, 1)
[ 1 -1 -1 -1 -1 -1 -1  1  1  1 -1  1 -1 -1 -1 -1  1  1  1 -1  1 -1  1 -1
 -1  1 -1 -1  1 -1  1 -1  1  1  1  1 -1 -1 -1  1 -1 -1 -1 -1  1 -1 -1 -1
  1 -1 -1 -1  1 -1  1  1 -1  1 -1  1  1 -1 -1 -1  1 -1 -1  1 -1 -1  1  1
 -1 -1  1 -1 -1  1  1 -1 -1  1 -1  1 -1 -1 -1 -1  1 -1  1 -1 -1 -1  1 -1
 -1  1 -1 -1 -1 -1 -1 -1 -1 -1  1  1 -1 -1  1  1  1  1 -1 -1 -1  1 -1 -1
 -1 -1  1  1 -1 -1 -1 -1 -1  1 -1  1 -1 -1  1 -1 -1 -1  1 -1  1 -1 -1  1
  1  1  1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1  1 -1 -1 -1 -1 -1  1  1  1 -1
 -1 -1 -1  1  1 -1 -1 -1 -1 -1 -1  1 -1  1 -1 -1  1 -1  1 -1  1 -1  1 -1
  1 -1  1  1 -1 -1  1  1 -1  1  1  1 -1  1  1  1 -1 -1  1  1 -1  1 -1  1
  1 -1  1  1 -1 -1  1 -1  1 -1 -1  1  1  1 -1 -1  1 -1 -1 -1 -1 -1 -1  1
 -1  1 -1 -1 -1  1  1 -1 -1  1 -1 -1  1 -1 -1  1 -1 -1 -1  1 -1 -1  1 -1
 -1 -1  1] 
 [-1  1 -1 ...  1 -1  1] 
 [ 1 -1 -1 -1 -1 -1 -1  1  1  1 -1  1 -1 -1 -1 -1  1  1  1 -1  1 -1  1 -1
 -1  1 -1 -1  1 -1  1 -1  1  1  1 -1 -1 -1 -1  1 -1 -1 -1 -1  1 -1 -1

## Step 8: Checking for Accuracy. 

In [26]:
v1 = 0  
v2 = 0
c = 0
i=0    
for c in range( np.size( target ) ) :  
  if y_test[c] == target[c] :            
    v1 = v1 + 1
  c = c +1
for i in range( np.size( t ) ) :
  if y_train[i] == t[i] :            
    v2 = v2 + 1
  i = i + 1
print("Accuracy of Defined Model on test data :",(v1/c)*100)     
print("Accuracy of Defined Model on train data:",(v2/i)*100)   

Accuracy of Defined Model on test data : 97.00374531835206
Accuracy of Defined Model on train data: 97.95663052543786
